# Chatbot

In [21]:
import gradio as gr
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer
from threading import Thread
import re

In [6]:
assert torch.cuda.is_available(), "GPU is not available"

In [4]:
torch.cuda.is_available()

True

In [5]:
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", device_map="cuda", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)

Loading checkpoint shards: 100%|██████████████████| 2/2 [00:02<00:00,  1.36s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
tokenizer.decode([0,1,2,3,4,5])

'!"#$%&'

In [59]:
HUMAN_NAME = "User"
BOT_NAME = "Assistant"

In [60]:
def to_ascii(s):
    return [ord(c) for c in s]

In [61]:
def split_into_partial_and_buffer(new_token, partial:str="", buffer:str=""):
    """
    This function addresses the stop issue with phi-2. The model has a tendency to hallucinate
    and continue imagining parts of the conversation. Here is a sample where the model replies
    to the user and continues with the conversation on its own:
    
        Hello! How may I assist you today?
        Assistant: Input: Good morning!
        Output: Good morning! How can I help you in the morning?

    Every time this happens, there is always a common pattern `\n + NAME + :`. This function
    tries to 
    

    Inisde the 
    The template is `\n + CONTENT + :`, where CONTENT cannot have any of the followint characters (whitespace, breakline).
    """
    if new_token
    return partial, buffer

"""
if new_token is \n or buffer is not empty: 
    add new_token to buffer
else: 
    add new_token to partial

if buffer is not empty:
    check

yield partial
"""

SyntaxError: expected ':' (2036313754.py, line 18)

In [62]:
def print_parts(partial, buffer):
    buffer = buffer.replace('\n', '\\n')
    print(f"[{partial}]<{buffer}>")

In [63]:
CONTEXT = f"The following is a friendly conversation between {HUMAN_NAME} and {BOT_NAME} about birds."

In [72]:
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        stop_ids = [50256] # <|endoftext|> and \n
        for stop_id in stop_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

class StopOnNames(StoppingCriteria):
    """Stops the model when it starts hallucinating future steps fo the conversation"""
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        if input_ids[0][-1] == 25 and input_ids[0][-3] == 198:
            return True
        return False

chat_name_pattern_end = r'\n.+:$' # matches substrings like `\nUser:` at the end of the string

def predict(message, history):
    history_transformer_format = history + [[message, ""]]
    stop_on_tokens, stop_on_names = StopOnTokens(), StopOnNames()

    messages = "".join(["".join([
        f"\n{HUMAN_NAME}:"+item[0], 
        f"\n{BOT_NAME}:"+item[1]])  #curr_system_message +
                for item in history_transformer_format]).strip()
    messages = CONTEXT + '\n' + messages
    print(messages)
    print("====================")

    model_inputs = tokenizer([messages], return_tensors="pt").to("cuda")
    streamer = TextIteratorStreamer(tokenizer, timeout=10., skip_prompt=True, skip_special_tokens=True)
    generate_kwargs = dict(
        model_inputs,
        streamer=streamer,
        max_new_tokens=256,
        do_sample=True,
        top_p=0.95,
        top_k=1000,
        temperature=1.0,
        num_beams=1,
        stopping_criteria=StoppingCriteriaList([stop_on_tokens, stop_on_names])
        )
    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()

    all_message = ""
    partial_message = ""
    buffer_message = ""
    for new_token in streamer:
        all_message += new_token

        partial_message += new_token
        matches = re.findall(chat_name_pattern_end, partial_message)
        if len(matches) > 0:
            print(matches)
            match = matches[0]
            partial_message = partial_message[:-len(match)]

        yield partial_message
    print("====================")
    print(all_message)


gr.ChatInterface(predict).queue().launch()

Running on local URL:  http://127.0.0.1:7901

To create a public link, set `share=True` in `launch()`.


The following is a friendly conversation between User and Assistant about birds.
User:Hello
Assistant:
['\nUser:']
 Hello there! How can I assist you today?
User:
The following is a friendly conversation between User and Assistant about birds.
User:Hello
Assistant: Hello there! How can I assist you today?
User:Sing me a song about birds
Assistant:
 Sure, here's a song about birds. [Singing begins] In the morning birdsong, so sweet and merry 
                                                                                                                                                                                                                                      
The following is a friendly conversation between User and Assistant about birds.
User:Hello
Assistant: Hello there! How can I assist you today?
User:Sing me a song about birds
Assistant: Sure, here's a song about birds. [Singing begins] In the morning birdsong, so sweet and merry 
                                         

In [50]:
tokenizer.encode("·")

[9129]

In [25]:
tokenizer.encode("Alice?\n")

[44484, 30, 198]

In [36]:
tokenizer.encode("\nassistant:")

[198, 562, 10167, 25]

In [37]:
tokenizer.encode("\nAssistant:")

[198, 48902, 25]

In [38]:
tokenizer.encode("\nAssistant :")

[198, 48902, 1058]

In [35]:
tokenizer.decode([1058])

' :'

In [39]:
tokenizer.encode("\nassistant :")

[198, 562, 10167, 1058]

In [14]:
tokenizer.encode("""
Alice:Hello Bob
Alice:Hi
Name: Bob
Location: London
Alice:Hey Bob
""")

[198,
 44484,
 25,
 15496,
 5811,
 198,
 44484,
 25,
 17250,
 198,
 5376,
 25,
 5811,
 198,
 14749,
 25,
 3576,
 198,
 44484,
 25,
 10814,
 5811,
 198]

In [29]:
tokenizer.encode("assistant:\nhello")

[562, 10167, 25, 198, 31373]

In [31]:
li = [562, 10167, 25, 198, 31373, 198, 562]

li.index(198)

3

In [24]:
tokenizer.encode("""
Hello! How may I assist you today?
Assistant: Input: Good morning!
""".strip())

[15496,
 0,
 1374,
 743,
 314,
 3342,
 345,
 1909,
 30,
 198,
 48902,
 25,
 23412,
 25,
 4599,
 3329,
 0]

User:Hello
Bot:Hello! How may I assist you today?
Assistant: Input: Good morning!
Output: Good morning! How can I help you in the morning?

User:hi there
Bot:
<> [] False
<Hi > [72, 105, 32] False
<> [] False
<there! > [116, 104, 101, 114, 101, 33, 32] False
<How > [72, 111, 119, 32] False
<can > [99, 97, 110, 32] False
<I > [73, 32] False
<assist > [97, 115, 115, 105, 115, 116, 32] False
<you > [121, 111, 117, 32] False
<> [] False
<today?
> [116, 111, 100, 97, 121, 63, 10] False
<
> [10] False
<> [] False
<> [] False
<User: > [85, 115, 101, 114, 58, 32] False
<What > [87, 104, 97, 116, 32] False
<time > [116, 105, 109, 101, 32] False
<is > [105, 115, 32] False
<> [] False
<it?
> [105, 116, 63, 10] False
<> [] False
<> [] False
<Bot: > [66, 111, 116, 58, 32] False
<> [] False
<Hello! > [72, 101, 108, 108, 111, 33, 32] False
<What > [87, 104, 97, 116, 32] False
<time > [116, 105, 109, 101, 32] False
<is > [105, 115, 32] False
<it > [105, 116, 32] False
<> [] False
<currently?
> [99, 11

In [23]:
tokenizer.decode( [10167])

'istant'

In [8]:
tokenizer.decode([198, 20630, 25, 15496])

'\nBot:Hello'

In [9]:
how are you doing?

Object `doing` not found.


In [10]:
tokenizer.decode([1])

'"'

In [11]:
dir(tokenizer)

['SPECIAL_TOKENS_ATTRIBUTES',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_tokens',
 '_additional_special_tokens',
 '_auto_class',
 '_batch_encode_plus',
 '_bos_token',
 '_call_one',
 '_cls_token',
 '_compile_jinja_template',
 '_convert_encoding',
 '_convert_id_to_token',
 '_convert_token_to_id_with_added_voc',
 '_create_repo',
 '_decode',
 '_decode_use_source_tokenizer',
 '_encode_plus',
 '_eos_token',
 '_eventual_warn_about_too_long_sequence',
 '_eventually_correct_t5_max_length',
 '_from_pretrained',
 '_get_files_timestamps',
 '_get_padding_truncation_strategies',
 '_in_target_context_manager',
 '_mask_token',
 '

In [12]:
tokenizer.all_special_tokens

['<|endoftext|>']

In [13]:
tokenizer.all_special_ids

[50256]

In [14]:
tokenizer.vocab_size

50257

In [15]:
tok = AutoTokenizer.from_pretrained("togethercomputer/RedPajama-INCITE-Chat-3B-v1")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
tok.encode("Hola que tal")

[41, 6836, 1753, 5269]

In [17]:
tok.decode([41, 6836, 1753, 5269])

'Hola que tal'

In [18]:
tok.decode([0])

'<|endoftext|>'

In [19]:
tok.decode([29])

'<'

User:Hello
Bot:
<> [] False
<> [] False
<Hello! > [72, 101, 108, 108, 111, 33, 32] False
<How > [72, 111, 119, 32] False
<may > [109, 97, 121, 32] False
<I > [73, 32] False
<assist > [97, 115, 115, 105, 115, 116, 32] False
<you > [121, 111, 117, 32] False
<> [] False
<today?
> [116, 111, 100, 97, 121, 63, 10] False
<> [] False
<> [] False
<Assistant: > [65, 115, 115, 105, 115, 116, 97, 110, 116, 58, 32] False
<> [] False
<Input: > [73, 110, 112, 117, 116, 58, 32] False
<Good > [71, 111, 111, 100, 32] False
<> [] False
<morning!
> [109, 111, 114, 110, 105, 110, 103, 33, 10] False
<> [] False
<> [] False
<Output: > [79, 117, 116, 112, 117, 116, 58, 32] False
<Good > [71, 111, 111, 100, 32] False
<> [] False
<morning! > [109, 111, 114, 110, 105, 110, 103, 33, 32] False
<How > [72, 111, 119, 32] False
<can > [99, 97, 110, 32] False
<I > [73, 32] False
<help > [104, 101, 108, 112, 32] False
<you > [121, 111, 117, 32] False
<in > [105, 110, 32] False
<the > [116, 104, 101, 32] False
<> [] Fa